# Correr experimentos
En este archivo está el código para correr los experimentos y escribir los resultados en archivos CSV.
> Los archivos se guardan en la carpeta _resultados_.

In [1]:
import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output

A continuación leemos los datasets en dataframes de Pandas.

In [2]:
def leer_instancia(path_instancia):
    with open(path_instancia, "r") as f:
        return f.read();


In [ ]:
# df_densidad_alta = pd.read_csv("instancias/densidad-alta/indice.csv");
# df_densidad_baja = pd.read_csv("instancias/densidad-baja/indice.csv");
# df_mejor_caso_bt = pd.read_csv("instancias/mejor-caso-bt/indice.csv");
# df_peor_caso_bt = pd.read_csv("instancias/peor-caso-bt/indice.csv");
# df_dinamica = pd.read_csv("instancias/dinamica/indice.csv");

In [3]:
df_mejor_caso_bt_f = pd.read_csv("instancias/mejor-caso-bt-f/indice.csv");
df_mejor_caso_bt_f.head(15)

,dataset,instancia,n,R,archivo
0,mejor-caso-bt-f,BT-MC-F-10,10,1000,instancias/mejor-caso-bt-f/BT-MC-F-10.txt
1,mejor-caso-bt-f,BT-MC-F-20,20,1000,instancias/mejor-caso-bt-f/BT-MC-F-20.txt
2,mejor-caso-bt-f,BT-MC-F-30,30,1000,instancias/mejor-caso-bt-f/BT-MC-F-30.txt
3,mejor-caso-bt-f,BT-MC-F-40,40,1000,instancias/mejor-caso-bt-f/BT-MC-F-40.txt
4,mejor-caso-bt-f,BT-MC-F-50,50,1000,instancias/mejor-caso-bt-f/BT-MC-F-50.txt
5,mejor-caso-bt-f,BT-MC-F-60,60,1000,instancias/mejor-caso-bt-f/BT-MC-F-60.txt
6,mejor-caso-bt-f,BT-MC-F-70,70,1000,instancias/mejor-caso-bt-f/BT-MC-F-70.txt
7,mejor-caso-bt-f,BT-MC-F-80,80,1000,instancias/mejor-caso-bt-f/BT-MC-F-80.txt
8,mejor-caso-bt-f,BT-MC-F-90,90,1000,instancias/mejor-caso-bt-f/BT-MC-F-90.txt
9,mejor-caso-bt-f,BT-MC-F-100,100,1000,instancias/mejor-caso-bt-f/BT-MC-F-100.txt


In [4]:
df_dinamica = pd.read_csv("instancias/dinamica/indice.csv");
df_dinamica.head(10)

,dataset,instancia,n,R,archivo
0,dinamica,DP-1000-1000,1000,1000,instancias/dinamica/DP-1000-1000.txt
1,dinamica,DP-1000-1500,1000,1500,instancias/dinamica/DP-1000-1500.txt
2,dinamica,DP-1000-2000,1000,2000,instancias/dinamica/DP-1000-2000.txt
3,dinamica,DP-1000-2500,1000,2500,instancias/dinamica/DP-1000-2500.txt
4,dinamica,DP-1000-3000,1000,3000,instancias/dinamica/DP-1000-3000.txt
5,dinamica,DP-1000-3500,1000,3500,instancias/dinamica/DP-1000-3500.txt
6,dinamica,DP-1000-4000,1000,4000,instancias/dinamica/DP-1000-4000.txt
7,dinamica,DP-1000-4500,1000,4500,instancias/dinamica/DP-1000-4500.txt
8,dinamica,DP-1000-5000,1000,5000,instancias/dinamica/DP-1000-5000.txt
9,dinamica,DP-1000-5500,1000,5500,instancias/dinamica/DP-1000-5500.txt


La siguiente función sirve para correr el código sobre una instancia ejecutando un método en particular.
- FB: Fuerza bruta
- BT: Backtracking con ambas podas.
- BT-F: Backtracking solamente con poda por factibilidad.
- BT-O: Backtracking solamente con poda por optimalidad.
- PD: Programación dinámica.

In [5]:
def correr_experimento(metodo, archivo_instancia):
    # Leer archivo de la instancia.
    instancia = leer_instancia(archivo_instancia)
    
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(["../main", metodo], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Poner la instancia en la entrada estandar.
    process.stdin.write(instancia)
    process.stdin.flush()

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise(F"Hubo un error en la experimentacion para el algoritmo: {algoritmo} con la instancia {archivo_instancia}.")
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    tiempo_de_ejecucion = float(process.stderr.read());
    
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    
    return tiempo_de_ejecucion;

## Corremos los experimentos
Vamos a guardar una tabla con las ejecuciones y sus respectivos tiempos.

In [6]:
experimentos = [];

## Experimento X
Correr BT-F para peor caso de poda de factibilidad.

In [6]:
for n in range(0, df_mejor_caso_bt_f.shape[0]):
    fila_n = df_mejor_caso_bt_f.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["R"], "BT-F", fila_n["archivo"]]);


### Experimento 1
Correr Fuerza Bruta en las primeras 30 instancias de densidad-alta y densidad-baja.

In [41]:
for n in range(0, 30):
    fila_n = df_densidad_alta.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "FB", fila_n["archivo"]]);

for n in range(0, 30):
    fila_n = df_densidad_baja.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "FB", fila_n["archivo"]]);

## Experimento 2
Correr BT para mejor caso y peor caso.

In [42]:
for n in range(0, df_mejor_caso_bt.shape[0]):
    fila_n = df_mejor_caso_bt.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "BT", fila_n["archivo"]]);

for n in range(0, df_peor_caso_bt.shape[0]):
    fila_n = df_peor_caso_bt.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "BT", fila_n["archivo"]]);

## Experimento 3
Correr BT, BT-F, BT-O para densidad alta y baja.

In [43]:
for n in range(0, df_densidad_alta.shape[0]):
    fila_n = df_densidad_alta.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "BT", fila_n["archivo"]]);
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "BT-F", fila_n["archivo"]]);
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "BT-O", fila_n["archivo"]]);

for n in range(0, 30):
    fila_n = df_densidad_baja.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "BT", fila_n["archivo"]]);
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "BT-F", fila_n["archivo"]]);
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "BT-O", fila_n["archivo"]]);

# Experimento J
Correr DP para el dataset _dinamica_.

In [8]:
for i in range(0, df_dinamica.shape[0]):
    fila = df_dinamica.iloc[i];
    experimentos.append([fila["dataset"], fila["n"], fila["R"], "PD", fila["archivo"]]);

## Experimento 5
Correr DP para el dataset densidad-alta y densidad-baja.

In [45]:
for i in range(0, df_densidad_alta.shape[0]):
    fila = df_densidad_alta.iloc[i];
    experimentos.append([fila["dataset"], fila["n"], fila["W"], "DP", fila["archivo"]]);

for i in range(0, df_densidad_baja.shape[0]):
    fila = df_densidad_baja.iloc[i];
    experimentos.append([fila["dataset"], fila["n"], fila["W"], "DP", fila["archivo"]]);

## Ejecutar los experimentos y guardar los resultados en un archivo CSV.
Este paso puede tardar unos minutos hasta terminar de ejecutarse.

In [ ]:
columnas = ["dataset", "n", "R", "metodo", "tiempo"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos = []
    resultados = []
    for i in range(0, T):
        t = correr_experimento(experimento[3], experimento[4])
        #t = exp[0]
        #res = exp[1]
        #resultados.append(res)
        tiempos.append(t);
    tiempo = np.median(tiempos);
    print(resultados)
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], tiempo]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado.csv", index=False, header=True);

'Experimento: 196/196'

In [ ]:
df_resultado